## Implementación de un modelo de lenguaje a nivel de caracteres

In [ ]:
import keras
import numpy as np
from keras import layers
from keras.models import Sequential

# en este ejemplo, se usa éste texto...
path = '../data/nietzsche.txt'
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


Definimos las secuencias, vocabulario y demás cosas para el modelo

In [ ]:
# Length of extracted character sequences
maxlen = 60
# We sample a new sequence every `step` characters
step = 3
# This holds our extracted sequences
sentences = []
# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


Ahora, considera éste modelo como el baseline

In [ ]:
model = Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               95232     
_________________________________________________________________
dense (Dense)                (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


Lo siguiente es entrenar el modelo, dar una secuencia inicial y generar texto a partir de esto. 
Una modificación para tratar de evitar la repetitividad de las secuencias generadas (sobre todo en las primeras épocas del entrenamiento), es modificar la distribución de probabilidad `softmax` para introducir cierta aleatoriedad en el proceso de muestreo (stochastic sampling). A partir de cierto parámetro ("temperatura"), se construye una nueva distribución de probabilidad de las salidas del modelo. El siguiente código implementa ésta idea.

In [4]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

El modelo de lenguaje queda entonces como

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.5, 1.0]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

### Generando tuits como AMLO

- Para los tweets, se realizó un pre-proceso, se pusieron todos los caracteres en minúsculas, se quitaron acentos, se quitaron caracteres duplicados, se cambiaron los Urls por la palabra url al final del tweet, se quitaron posibles emojis o referencias a otros usuarios (@user).

- Para el modelo, se probó el baseline con 5 epocas, diferentes optmizadores (RMSprop, SGD, Adam), diferentes tamaños de learning rate (0.001, 0.01, 0.1), diferentes tamaños de minibatch (8-512). Posteriormente se probó con diferente numero de unidades en la capa LSTM (64-256), agregando mas capas LSTM y probando con capa bidireccionales y por ultimo dejando entrenar el modelo hasta obtener una buena generación de texto.

- El mejor modelo resultó entrenando por 40 epocas, con optimizador RMSprop, con learning rate 0.01, con tamaño de minibatch de 256, con capa LSTM con 256 unidades y una capa densa con el tamaño de numero de caracteres y función de activación softmax, con un error de entrenamiento de 0.71


In [7]:
# librerias
import keras
import numpy as np
from keras import layers
from keras.models import Sequential

import tensorflow as tf
import random

#path
path = "/home/victor/cursos/banxico/nlp_2020/data/" #path de los archivos

#tweets
text = open(path+"amlo_tuits_preprocesados.txt").read().lower() #carga los tweets procesados

#modelo
Modelo1 = tf.keras.models.load_model(path+'amlo_tuits.h5', compile=False) #carga el mejor modelo
Modelo1.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_39 (LSTM)               (None, 256)               319488    
_________________________________________________________________
dense_36 (Dense)             (None, 55)                14135     
Total params: 333,623
Trainable params: 333,623
Non-trainable params: 0
_________________________________________________________________


In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#Función para generar un tweet de AMLO
def generate_AMLO_tweet(model=Modelo1, text=text, temperature=0.3):
  maxlen=60
  chars = sorted(list(set(text))) #vocabulario o caracteres unicos
  char_indices = dict((char, chars.index(char)) for char in chars) #crea un diccionario donde las llaves son los caracteres y los valores son los indices
# Select a text seed at random
  start_index = random.randint(0, len(text) - maxlen - 1) #selecciona del texto original aleatoriamente una secuencia del mismo tamaño con el que se entrenó
  generated_text = text[start_index: start_index + maxlen]
  
  # We generate 140 characters
  i=0
  next_char=''
  tweet_char=[]
  tweet_char.append(generated_text)
  while i<140 and next_char != '\n': #genera 400 caracteres
    sampled = np.zeros((1, maxlen, len(chars))) #la semilla la vectoriza
    for t, char in enumerate(generated_text):
      sampled[0, t, char_indices[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0] #predice con la semilla y obtiene la distribucion de probabilidad
    next_index = sample(preds, temperature) #obtiene el siguiente indice del caracter
    next_char = chars[next_index] #obtiene el caracter
    i+=1
    tweet_char.append(next_char)
    generated_text += next_char #concatena el caracter
    generated_text = generated_text[1:] #da un paso adelante para actualizar la entrada para la proxima prediccion
  tweet="".join(ch for ch in tweet_char)  #concatena
  print(tweet) #muestra el tweet

Finalmente se probó con diferentes valores de temperatura (0.2-3), notando que:

- Temperatura <0.3 se repetían mucho algunas palabras
- 0.3< Temperatura <0.5 se obtenían mejores tweets
- 0.5< Temperatura <1 se obtenían algunas palabras raras
- 1< Temperatura,  no tenía ningún sentido

Algunos ejemplos con temperatura=0.3:

In [12]:
print('AMLO Tweets:')
print()
for i in range(10):
  generate_AMLO_tweet(temperature=0.3)
  print()

AMLO Tweets:

 tlapacoyan y por la noche en misantla veracruz por eso la gente en el pais no legado se responde a seguir en el pacifica la entrevista que me hizo ciro por los empresarios que impera en chihuahua y l

an lazaro cardenas zihuatanejo y vamos de regreso a 20 dias de chiapas en el problema de corupcion para el bienestar de las ciastandos de mexico url


es tiera fertil para el genocidio ni para canalas que lo impuesto por la redes social url


l a la reforma educativa en toluca se presento a la maestra delfina gomez y trataremos la presidencia en el acto de ser credito presente a la notria de gobierno de la solicitud provotica en el preside

ion de registro del partido de calderon url
 seguiremos cumplia el gobernador de la solidaridad y el presidente de mexico a la gente en el problema de la frontera y aliandre y el problema de la solida

 mercado a nadie engana y puede resultar aterador repudio la corupcion y el prd y los de la madada del prd url


omplejo petroquimico paj